# MMSBR preprocess step 1

In [ ]:
import gzip
from collections import defaultdict
from datetime import datetime
import os
import time
import pickle
import pandas as pd
import json as json
import numpy as np
from tqdm import tqdm

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

meta_dataset = 'Clothing_Shoes_and_Jewelry'
# Grocery_and_Gourmet_Food/Electronics/Home_and_Kitchen
# datasets_name = meta_dataset + '_5'

price_level_num = 99
time_interval = 60*60*24
data_path = 'originalData/meta_' + meta_dataset + '.json.gz'
df_item = getDF(data_path)


# item attributes, itemID-price-category-title-imageURL-imageURLHigh
item_property = df_item[['asin', 'price', 'category', 'title', 'imageURL', 'imageURLHighRes']]

item_property = item_property.dropna(axis=0)

# filtering out items with invalid text or images
if meta_dataset == 'Cell_Phones_and_Accessories':
    item_property = item_property[item_property['asin'] != 'B0187XZOUC']
    item_property = item_property[item_property['asin'] != 'B00M4G42W0']
elif meta_dataset == 'Grocery_and_Gourmet_Food':
    # item_property = item_property[~item_property['asin'].isin(stopItem)] 
    pass
elif meta_dataset == 'Sports_and_Outdoors':
    stopItem = ['B00R58SRYQ', 'B00E9A52YQ', 'B00HPT7IF4', 'B004EPXXQU', 'B004ELBRK8', 'B00DVOLLKA', 'B011RTEN8C', 'B00PBEHHAG', 'B009TNKXRO', 'B00CP6AML4', 'B00E1EU0Q0', 'B00RHA8O9K', 'B00AG270IM', 'B00P8C789Q', 'B00CN9OXWC', 'B004JYNAAA', 'B00NX46FX6', 'B00NX46XU6', 'B003BLOTX4', 'B00LFITP9S', 'B00DDTI29Q', 'B008B9D2ZC', 'B00MMTRHGW', 'B00MMTXAEK', 'B00DVNBTWG', 'B00914S9GI', 'B001IHERIA', 'B00RH9XOCI', 'B00L1R8Y72', 'B00BC1ZVP0', 'B01HEGK1S8', 'B00R7AXISW', 'B003BLOUCO', 'B0049MHMEC', 'B004J2S3I6', 'B00R59X3QM', 'B00MWF4B9M', 'B00BPIDGFM', 'B005W3C9C2', 'B00HE4PS2U', 'B00KNE5VGQ', 'B00L4IGV3M', 'B00L4IGRR2', 'B00TB0BL94', 'B00TB0MWTM', 'B01HEGJX4Q', 'B01HEGK5U2', 'B008I7Q81W', 'B0108NM3SA', 'B001154PLI', 'B00LFJFUMS', 'B01AANWPWM', 'B00P8DQZHQ', 'B00DIIS346', 'B00FSAJFUI', 'B00DVOKBW4', 'B00R58UIW0', 'B00EQ6OSOI', 'B0147JS4DO', 'B0147JSTUC', 'B0155UIGSM', 'B00QH6LH4O', 'B019Z7T8QK', 'B019Z7THLG', 'B0054PD0SU', 'B00KZHR24A', 'B00G7LEEXK', 'B00QVZJUMM', 'B00QVB3EIC', 'B01FXOF0CU', 'B001MS5NNI', 'B001MS9EYM', 'B018RK85F6', 'B00K5T5FN8', 'B00F1JVW0W', 'B01HEGJX3C', 'B00AYS9LGC', 'B00ONACPR4', 'B00DDTZDXY', 'B002PN2RA6', 'B01HEGK5TI', 'B00OVTC4VE', 'B00LFIQQUY', 'B00JRVW7F4', 'B0067VNG6Q', 'B00KIXW42Q', 'B0012QWACW']
    item_property = item_property[~item_property['asin'].isin(stopItem)] 
elif meta_dataset == 'Clothing_Shoes_and_Jewelry':
    stopItem =['B00009UW4R', 'B0006Z8AYM', 'B0006Z8AYC', 'B0007TVDY6', 'B0008IW19C', 'B000BYM2JC', 'B000C1U22S', 'B000CODPNI', 'B000FDY8RS', 'B000HAOT50', 'B000KLV1CA', 'B000KLSYH0', 'B000NDCHPA', 'B000OWCS6I', 'B000RGJOMW', 'B000RLFHO6', 'B000SG7D8I', 'B000X172R0', 'B000XZDOQY', 'B0011FU3NC', 'B0012PQWAY', 'B0015BT35G', 'B001B7HSV0', 'B001HZEKIA', 'B001N2ZN74', 'B001N3354G', 'B001N2XX6M', 'B0029LIAXC', 'B002B55GL0', 'B002ETVLZ8', 'B002GYW30I', 'B002KPHXP8', 'B002M62RI8', 'B002R5YZC0', 'B002SWJBNU', 'B002VPETUE', 'B002ZGECMO', 'B0032XDS56', 'B0033ESTJ4', 'B002RV7OK4', 'B00368C8LC', 'B0037TMAZY', 'B0037TTOA8', 'B0038HYIPU', 'B003B2SVHS', 'B003DQN8JI', 'B003N9BV7Q', 'B003PLKFNS', 'B003S1U1CK', 'B003S0NGAU', 'B003UE5I5A', 'B003WOKYW0', 'B003XKNB14', 'B00400N72O', 'B0042VIJOW', 'B0047Y1OSM', 'B0048W5IHQ', 'B004D2RH6G', 'B0059P8SJG', 'B0059RLE8G', 'B005AOIEG8', 'B005B7U0LG', 'B005FKNHNC', 'B005GK97MG', 'B005M9JALO', 'B005PFPSBQ', 'B005RS4ZFG', 'B005XJYJ5K', 'B006JH6XF4', 'B006OC0SQO', 'B0072D7I7G', 'B0072H5QIU', 'B0077HZ3T2', 'B007TLEZUY', 'B007TJFHI0', 'B007U1ULRY', 'B008DVKVZC', 'B008EYPC70', 'B008FV74N2', 'B008GXVXY0', 'B008HU9786', 'B008PDBVQG', 'B008PDBX0U', 'B008YWFBXC', 'B0091CYCHU', 'B00946JATI', 'B0098AAYAO', 'B009GPIE9E', 'B009L4CSYC', 'B009NFFWRY', 'B00A4CXBAU', 'B00AJLJEEI', 'B00AN53HR0', 'B00AWB15MY', 'B00B6EA0EK', 'B00B6E8RKO', 'B00B78BB5C', 'B00BV2VUFA', 'B00BV2VUJ6', 'B00BZDTHBY', 'B00C6TZZXA', 'B00C9T0KU0', 'B00C9WJ6HK', 'B00CDBT834', 'B00CIJ8YGS', 'B00CLVLEM4', 'B00CMNTRUW', 'B00CN7TRE8', 'B00CPKE4EG', 'B00CTA12QA', 'B00CUK97DY', 'B00CULPGZ6', 'B00CUM1EMY', 'B00CUM2I1A', 'B00CXMYAYU', 'B00CXQFOXW', 'B00CZDU0PA', 'B00D9JQHQU', 'B00DB1TK4C', 'B00DFQYEBW', 'B00DG7EQ1I', 'B00DG7GPCQ', 'B00DNNX4VI', 'B00DNO3VDS', 'B00DOZM916', 'B00DQYXJLO', 'B00DU0OBKM', 'B00DW7MIYE', 'B00DWZH8GE', 'B00DYAYVLW', 'B00E1ZPPXC', 'B00E253Z74', 'B00E41Z7YU', 'B00E4MPTME', 'B00E56L9MS', 'B00E5RKBV2', 'B00E8GRAFU', 'B00E8GR8IY', 'B00E8GR54Q', 'B00E9EG05M', 'B00EPYCHTO', 'B00ESXEBYG', 'B00EUBKRGM', 'B00EVTLF6Y', 'B00EXJ2LGA', 'B00EYY755Q', 'B00F1X6BLS', 'B00F3AWCVW', 'B00F6K7V5Q', 'B00FAYHW34', 'B00FD2MC1K', 'B00FL79NUU', 'B00FMIW7PQ', 'B00FMMWGAI', 'B00FMMW84C', 'B00FWZQV74', 'B00G0ON9XU', 'B00G3YBKJC', 'B00G3YBDK8', 'B00G4C25BA', 'B00G7N0TLE', 'B00GJATZDI', 'B00GMDIXGC', 'B00GMEVE26', 'B00GO98GZW', 'B00GW7ZOGA', 'B00GYX3BGW', 'B00H2SPZPI', 'B00HD51JGO', 'B00HD6N3B2', 'B00HD6MPHK', 'B00HG83L3W', 'B00HJ2GQH8', 'B00HO63M80', 'B00HQOAVUW', 'B00HVVK0RE', 'B00HZFAORM', 'B00I1AEA8Y', 'B00I1AE6Z6', 'B00I1AHCZM', 'B00I4KXLTA', 'B00I6CZQNA', 'B00IGQTYLQ', 'B00IJXLJ14', 'B00IKDRL8S', 'B00IKDRIIQ', 'B00IS5BTOU', 'B00IVGHASU', 'B00IVLMVIE', 'B00IXKP2O8', 'B00J2UNCEK', 'B00JA4KXRC', 'B00JA8EDUG', 'B00JB5RPGM', 'B00JB8ORSS', 'B00JB8PBN8', 'B00JB8PSMW', 'B00JB8OYYU', 'B00JB8PECG', 'B00JBXOS64', 'B00JC10A1M', 'B00JFTVHWW', 'B00JGEH23O', 'B00JGWCHYA', 'B00JH9Q7OI', 'B00JHDTAWK', 'B00JVVVAKI', 'B00K67BOW0', 'B00K69LYR8', 'B00KDJ8PFU', 'B00KDXU2MU', 'B00KMOSDKI', 'B00KMVI69O', 'B00KPPOIY4', 'B00KQ3OFTS', 'B00KQHF3IG', 'B00KR9SEO8', 'B00KS19POI', 'B00KT8TK9U', 'B00KVTSPRU', 'B00KVU1UPS', 'B00KZMBEY4', 'B00L18PJY2', 'B00L2ON49W', 'B00L2ITI4S', 'B00L3KVUC8', 'B00L4YBUPA', 'B00L9TQQHM', 'B00LB1VAAQ', 'B00LE6J3VQ', 'B00LEOB2NU', 'B00LHZFSWW', 'B00LHZFOD0', 'B00LIPAJGG', 'B00LIRYZSM', 'B00LIS0TC2', 'B00LIY23XO', 'B00LKY0B04', 'B00LN8Q8Y0', 'B00LOB6BFW', 'B00LO9Y4OO', 'B00LOB5X74', 'B00LU5712O', 'B00LUXLFG4', 'B00M0DGUI6', 'B00M0XW472', 'B00M28VJFI', 'B00M3KKZYG', 'B00M4WQ7FY', 'B00M4WP4JY', 'B00M590TQE', 'B00M8YX68Y', 'B00M8YXOSQ', 'B00MCKVDNY', 'B00MF0RS1W', 'B00MI420IQ', 'B00MIU8TSK', 'B00MNJXQ8Y', 'B00MOUBHCY', 'B00MPR7X9C', 'B00MPR7T7I', 'B00MTFKZ2C', 'B00MTFLBHK', 'B00MY0DUGU', 'B00N3RUS68', 'B00N4XYTNY', 'B00NAU2DA2', 'B00NAWKASM', 'B00NEAGTV2', 'B00NEAH0JC', 'B00NF26NKQ', 'B00NIN6LUY', 'B00NIN2NZQ', 'B00NIMYVIE', 'B00NIMYNQE', 'B00NL0TICW', 'B00NLAJ6IS', 'B00NL0TMUU', 'B00O4QYZLQ', 'B00O61DALK', 'B00O61EFOG', 'B00OAN1VTM', 'B00OJ3W8FE', 'B00OK7AETG', 'B00OABK668', 'B00OLOYY26', 'B00OV4EPGG', 'B00OVFYACY', 'B00OVUBOG4', 'B00P7PCL3M', 'B00PDJS1K4', 'B00PUA2TTK', 'B00PUDWWBM', 'B00PY32SBM', 'B00PY4WG0O', 'B00QBINT8A', 'B00QBUZN0K', 'B00QBV53BS', 'B00QV9HVY2', 'B00RD5RQG6', 'B00RD5SDAO', 'B00RDDSQIU', 'B00RET8CT6', 'B00RVLOXKE', 'B00RYCFMJQ', 'B00RZPXWGW', 'B00S2SSG6W', 'B00S9NP6UE', 'B00SSAD9ZM', 'B00STAL4R6', 'B00T46UE7A', 'B00T6IE9FO', 'B00TATGZ74', 'B00TATH9WY', 'B00TIE855Q', 'B00TIRMCDY', 'B00U36B66I', 'B00U8UYEQI', 'B00U8KZRNM', 'B00UB1URNI', 'B00UNIDMOU', 'B00UOV2HXI', 'B00UVPSFRO', 'B00UXWXE2Q', 'B00V0G98FQ', 'B00V5Y7PLW', 'B00V8ZPFHE', 'B00VQQU4ZS', 'B00VR20BUY', 'B00VSJ40HG', 'B00VWW0L2M', 'B00WDWBBRY', 'B00WGV3SZK', 'B00WHXNEZQ', 'B00WIS7FD2', 'B00WM3JV80', 'B00WSVP4KK', 'B00WV1UGVO', 'B00WWFR7RU', 'B00WWFS7PQ', 'B00WWYQFA6', 'B00WYP39IO', 'B00X5TVQ56', 'B00X5TZS3W', 'B00X5WMBZM', 'B00X5WLFYK', 'B00X5U0GMO', 'B00XH7I2WQ', 'B00XKXIAAG', 'B00XKXI3J4', 'B00XKXZIPG', 'B00XKY9SEW', 'B00XKY84LK', 'B00XKYS2YE', 'B00XKXHMPU', 'B00XLQE5J2', 'B00XMN717M', 'B00XO34N30', 'B00Y4RF9EW', 'B00Y4RHIRI', 'B00Y7WYPHQ', 'B00YE0OU8U', 'B00YE0PU6G', 'B00YHCLD8U', 'B00YJ34K9G', 'B00YOYIMT4', 'B00YQAF4GA', 'B00YQYZZKQ', 'B00YX4F6MQ', 'B00Z5SM9RE', 'B00Z9AMQOO', 'B00Z9Y360S', 'B00ZFZ5FV4', 'B00ZHONSD0', 'B00ZKDYFLM', 'B00ZN3OVPO', 'B00ZP9XN00', 'B00ZUNFCVE', 'B01001ZUYY', 'B01015RZVK', 'B01028592I', 'B0105IFL9Q', 'B010824FRI', 'B010D2MB4C', 'B010NZ27IY', 'B010RRVO7I', 'B010S61XZG', 'B010S5F2RM', 'B010VXVC3O', 'B010VXVYVO', 'B011354WXG', 'B01167U2NA', 'B0117EN8FG', 'B01192N4AU', 'B0119ERG0C', 'B0119YMEOA', 'B011HPZMFO', 'B011ISHNW0', 'B011JDBQOA', 'B011NV7VPQ', 'B011R59YYY', 'B011RT6QR8', 'B011ST202M', 'B011YKPVQC', 'B011Z7D67A', 'B011Z7CMZW', 'B0124GGXUI', 'B0124H2FTK', 'B0127TX25Q', 'B0127WIO6A', 'B012AW9UAG', 'B012AWCYZO', 'B012E0E1QC', 'B012FE487Y', 'B012I5A6XK', 'B012I5B10M', 'B012I5IUB0', 'B012IUCCM8', 'B012OUSEHE', 'B012OUT3FG', 'B012OURH0Y', 'B012OUSK4G', 'B012OURN9E', 'B012P2G9I2', 'B012P2H01C', 'B012TJCMZU', 'B012TJG9I6', 'B012TJCVMO', 'B012TJGOY0', 'B012TJPR20', 'B012TJCNKO', 'B012TJDAGU', 'B012TJEB5Y', 'B012TJDI6C', 'B0131JSE1I', 'B0131JSP62', 'B0131JMSCE', 'B0132IOJPI', 'B0132K6ADA', 'B0132JEAZ6', 'B013381K2C', 'B0132K8X08', 'B0133J7X8G', 'B0133Z4BV2', 'B0137DUZP6', 'B013AW8XJO', 'B013AW9A5A', 'B013BSMAWS', 'B013G0W1QQ', 'B013H1SBBS', 'B013H4XAO8', 'B013HDY47Q', 'B013I4IFH4', 'B013I4INZ8', 'B013I4IKPG', 'B013I4HY3K', 'B013I4ICOA', 'B013KCZS6A', 'B013KG0UV4', 'B013LJYY12', 'B013O8X2Q8', 'B013O9MY1Q', 'B013OVJZBQ', 'B013QM6EGC', 'B013QM7D4E', 'B013QM6LSI', 'B013RMOICI', 'B013WBIYEC', 'B0143ORPVK', 'B0143OREHA', 'B0143OS36Q', 'B0144EANT0', 'B0147H9SAE', 'B0147H9DSG', 'B0147H9T4E', 'B0147H96YC', 'B014C0C07Y', 'B014MGNCC0', 'B014RPHIWQ', 'B014STNXWK', 'B014USYCFQ', 'B014WMCC02', 'B014WMCJWI', 'B0151880AW', 'B0151JVSRI', 'B0151XCNTQ', 'B0151XEAJM', 'B0153B0UFK', 'B0153B0J56', 'B0158PWOZA', 'B015GAQFZ2', 'B015GI8SQI', 'B015M163FC', 'B015NKT1VU', 'B015O1D560', 'B015XDRWYU', 'B015ZCY7FG', 'B01620C2HK', 'B0169VPD6E', 'B0169VOXXS', 'B016I7IZ6E', 'B016KL6TXY', 'B016PHFM4A', 'B016S45MRW', 'B0172HKWR4', 'B0172HLJJE', 'B01750F2AK', 'B017BCFIW4', 'B017BZZNWQ', 'B017BZYII6', 'B017C49HXW', 'B017CCAKA8', 'B017JPQ426', 'B017P0F534', 'B017UFJKY4', 'B017URS50W', 'B017Y1GTSE', 'B017Y1GOZ2', 'B0184L03O4', 'B0187DU1XM', 'B018EVB87U', 'B0196BZ8HS', 'B019QYCOW2', 'B019S0TFYE', 'B019WVMVAO', 'B019X17J4G', 'B019X2V6AI', 'B019YMPHYS', 'B01A86MPTE', 'B01AIEJEZE', 'B01AMKUIJA', 'B01AUJSJYY', 'B01AW5LDEE', 'B01B24SH2A', 'B01B48O3U4', 'B01B5OGJE0', 'B01B7H802Y', 'B01B8J5VNW', 'B01BGYHH1S', 'B01BGYH5GK', 'B01BMPYN7W', 'B01BTEF8KC', 'B01BTEFQKO', 'B01CKHV2NE', 'B01E0FQPNG', 'B01E0FRHSS', 'B01E0FQRZM', 'B01E2S8KNO', 'B01E7QQHRW', 'B01E9FRWAW', 'B01E9GP8RK', 'B01EJW36QY', 'B01EXD5CLQ', 'B01EY7HUU2', 'B01FE7JP4A', 'B01FRLDFY4', 'B01FUQQMA0', 'B01G61T2X2', 'B01GEGLEIK', 'B01GEGLBDI', 'B01GEGLK98', 'B01GEGL8Z4', 'B01H3EBIFG', 'B01H3EBD1A', 'B01H3EFDDO', 'B01H43GVK8', 'B01H43JUQK', 'B01H7HON9W', 'B01HC53JJ4', 'B01HC535O8', 'B01HHCC7I6'] 
    item_property = item_property[~item_property['asin'].isin(stopItem)]
    
def is_number(s):
    
    try:
        float(s)
        return True
    except ValueError:
        pass
    if '−' in  s:
        return True
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False


def reg_price(price):
    if is_number(price):
        return  float(price)
    if '−' in price:
        price_list = price.replace(' ', '').split('−')
        results = (float(price_list[0]) + float(price_list[1]))/2
        return results
    price_num = price.replace('$','')
    if is_number(price_num):
        results = float(price_num)
    else:
        results = ''
    return results

def reg_category(cate):
    results = ''
    if isinstance(cate,list):
        if len(cate) == 0:
            results = ''
        else:
            results = cate[-1]
    else:
        results = ''
    return results

def get_text(title):
    # text = title 
    results = ''
    if title != '':
        results=[]
        results.append(title)
    return results

def get_image(image):
    results = ''
    if isinstance(image,list):
        if len(image) != 0:
            results = image[0]
    return results

item_property['price_num'] = item_property.price.map(reg_price)
item_property['cate'] = item_property.category.map(reg_category)



item_property['text'] = item_property.apply(lambda row: get_text(row['title']), axis=1)
item_property['image'] = item_property.apply(lambda row: get_image(row['imageURLHighRes']), axis=1)



# delete items without corresponding attributes
item_property = item_property[(item_property['price_num'] != '')]
item_property = item_property[(item_property['cate'] != '' )]
item_property = item_property[item_property['text'] != '']
item_property = item_property[(item_property['image'] != '' )]

item_property[['price_num']] = item_property[['price_num']].astype(float)

item_property.drop_duplicates(subset=['asin'],keep='first',inplace=True)


item_data = item_property[['asin', 'price_num', 'cate', 'image', 'text']]

group_cate_num = pd.DataFrame(item_data.groupby(item_data['cate']).count())
group_num=group_cate_num.reset_index()[['cate','asin']].rename(columns={'asin':'count'})

group_cate_min = pd.DataFrame(item_data['price_num'].groupby(item_data['cate']).min())
group_min=group_cate_min.reset_index()[['cate','price_num']].rename(columns={'price_num':'min'})

group_cate_max = pd.DataFrame(item_data['price_num'].groupby(item_data['cate']).max())
group_max=group_cate_max.reset_index()[['cate','price_num']].rename(columns={'price_num':'max'})

group_cate_mean = pd.DataFrame(item_data['price_num'].groupby(item_data['cate']).mean())
group_mean=group_cate_mean.reset_index()[['cate','price_num']].rename(columns={'price_num':'mean'})

group_cate_std = pd.DataFrame(item_data['price_num'].groupby(item_data['cate']).std())
group_std=group_cate_std.reset_index()[['cate','price_num']].rename(columns={'price_num':'std'})

item_data1 = pd.merge(item_data, group_num, how='left', on = 'cate')
item_data2 = pd.merge(item_data1, group_min, how='left', on = 'cate')
item_data3 = pd.merge(item_data2, group_max, how='left', on = 'cate')

item_data4 = pd.merge(item_data3, group_mean, how='left', on = 'cate')
item_data = pd.merge(item_data4, group_std, how='left', on = 'cate')


item_data = item_data[item_data['count']>4]

item_data = item_data[item_data['std']!=0]

item_data = item_data[['asin', 'price_num', 'cate', 'image','text', 'min', 'max', 'mean', 'std']]

import math
def get_price_level(price, p_min, p_max):
    fenzi = price - p_min
    fenmu = p_max - p_min
    if fenmu == 0 or price==0:
        return -1
    results = int(fenzi / fenmu * price_level_num) + 1
    return results

def logistic(t, u, s):
    gama = s * 3**(0.5) / math.pi
    results = 1 / (1 + math.exp((t-u)/gama))
    return results

item_data['price_level'] = item_data.apply(lambda row: get_price_level(row['price_num'], row['min'], row['max']), axis=1)
item_final = item_data[item_data['price_level'] != -1]
item_final = item_final[['asin', 'price_num', 'cate', 'image','text', 'price_level']]
item_final = item_final.reset_index(drop=True)

print("done")

In [ ]:

# dict item_id: image_url
id2image = {}
# dict item_id: text
id2text = {}

for _, row in tqdm(item_final.iterrows()):
    if row['asin'] not in id2image:
        id2image[row['asin']] = row['image']
    if row['asin'] not in id2text:
        id2text[row['asin']] = row['text']

dict_path = './dict/' + meta_dataset
if not os.path.exists(dict_path):
    os.makedirs(dict_path)
dict_ID2text = dict_path + '/id2text.csv'
id2text_key_list = list(id2text.keys())
id2text_val_list = list(id2text.values())
id2text_dict = pd.DataFrame({'asin':id2text_key_list, 'text':id2text_val_list})
id2text_dict.to_csv(dict_ID2text)

# id2image = {'4639725043': 'https://images-na.ssl-images-amazon.com/images/I/31V3NfjNACL.jpg'}
# id2text = {'4639725043': [text]}
dict_path = './dict/' + meta_dataset    
dict_save_path = dict_path + '/item_final.csv'
dict_image_save = dict_path + '/id2imageURL.npy'
dict_text_save = dict_path + '/id2text.npy'
if not os.path.exists(dict_path):
    os.makedirs(dict_path)
item_final.to_csv(dict_save_path)
np.save(dict_image_save, id2image)
np.save(dict_text_save, id2text)
print("dataset: ", meta_dataset, "save the meta and dict")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

In [ ]:
import gzip
from collections import defaultdict
from datetime import datetime
import os
import time
import pickle
import pandas as pd
import json as json
import numpy as np

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

meta_dataset = 'Clothing_Shoes_and_Jewelry'
datasets_name = meta_dataset
interaction_path = './originalData/' + datasets_name + '_5.json.gz'
# df = getDF('originalData/Tools_and_Home_Improvement.json.gz')
df = getDF(interaction_path)
time_interval = 60*60*24

temp_iter = df[['reviewerID', 'asin', 'unixReviewTime']]

def merge_review_time(review, time):
    return str(review)+str(time)

temp_iter['review_time'] = temp_iter.apply(lambda row: merge_review_time(row['reviewerID'], row['unixReviewTime']), axis=1)
temp_iter = temp_iter[['review_time', 'asin', 'unixReviewTime']]
interaction=temp_iter.rename(columns={'review_time':'reviewerID'})
interaction.head(5)

In [ ]:
item_path = './dict/' + meta_dataset + '/item_final.csv'
item_final =  pd.read_csv(item_path)
item_final = item_final[['asin', 'price_level', 'cate']]


user_item1 = pd.merge(interaction, item_final, how='left', on = 'asin')
user_item2 = user_item1.dropna(axis=0)


user_item2.sort_values(by=["reviewerID","unixReviewTime"],inplace=True,ascending=[True,True])
user_click_num = pd.DataFrame(user_item2.groupby(user_item2['reviewerID']).count())
click_num=user_click_num.reset_index()[['reviewerID','asin']].rename(columns={'asin':'click_num'})
item_data6 = pd.merge(user_item2, click_num, how='left', on = 'reviewerID')
item_data7 = item_data6[item_data6['click_num']>1]
data = item_data7[['reviewerID', 'asin', 'unixReviewTime', 'price_level', 'cate']]
data.head(5)

In [ ]:
data2 = data

user_click_num2 = pd.DataFrame(data2.groupby(data2['reviewerID']).count())
click_num2=user_click_num2.reset_index()[['reviewerID','asin']].rename(columns={'asin':'click_num'})
data3 = pd.merge(data2, click_num2, how='left', on = 'reviewerID')
data4 = data3[data3['click_num']>1]



save_path = './dict/' + meta_dataset +  '/interaction.csv'
data4.to_csv(save_path)
print("dataset: ", meta_dataset)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))